***Webscraper Hotels Data - 123 Milhas / Decolar***

*   Data extraction name, price and provider hotels website 123 Milhas
*   Data extraction name and price hotels website Decolar
*   Rate comparison
*   Search details


In [1]:
!pip install selenium
!pip install unidecode
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 11.0 MB/s 
     |████████████████████████████████| 359 kB 17.1 MB/s 
     |████████████████████████████████| 138 kB 84.1 MB/s 
     |████████████████████████████████| 4.0 MB 59.9 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in ind

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from unidecode import unidecode
from time import sleep
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
import gspread
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
import gspread_dataframe as gd

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# Chromedriver config

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
def rate_fmt(rate):
    rate = ' '.join(rate)
    rate = rate.strip('R$').lstrip().replace(',','.')
    rate = float(rate)
    return rate

In [5]:
pax = 2 # Number of guests (adults)

In [6]:
destinations = {
  'name_destination' : ['Aracaju','Arraial dAjuda','Balneario Camboriu','Belo Horizonte','Brasilia','Buzios','Cabo Frio','Caldas Novas','Campinas','Campos do Jordao','Canela','Curitiba','Florianopolis','Fortaleza','Foz do Iguacu','Gramado','Ilhabela','Ilheus','Joao Pessoa','Maceio','Morro de Sao Paulo','Natal','Porto Alegre','Porto de Galinhas','Porto Seguro','Praia de Pipa','Recife','Rio de Janeiro','Salvador','Sao Paulo'],
  'id_destination_123milhas' : ['1010804','1184784','1013062','1017081','1262440','1156720','1027418','1027035','1265185','1621906','1006401','1112002','1001872','1001925','1106867','1006404','1012575','1002846','1111236','1009657','1060292','1009933','1112012','1024069','1262144','1060267','1010394','1010422','1010693','1010502'],
  'id_destination_decolar' : ['180','339','192594','701','926','1077','1227','1364','1467','6116','6064','1595','2261','2302','3072','2611','3027','3158','3399','4430','4678','4971','5822','5648','881','5875','6322','6381','7018','6574']
}
dates = {
  'checkin' : ['29-06-2022'],
  'checkout' : ['30-06-2022']
}

In [7]:
# Get fee 123 Milhas

def get_fee():
    global fee
    fee = 0

    c_dest = 0
    c_date = 0

    while fee == 0:
      try:
        driver = webdriver.Chrome(options=chrome_options)
        link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={dates["checkin"][c_date]}&checkout={dates["checkout"][c_date]}&criancas=0&id={destinations["id_destination_123milhas"][c_dest]}&localizacao={destinations["name_destination"][c_dest]},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
        driver.get(link)
        sleep(3)
          
        price_net_temp = []
        tax_temp = []

        wait = WebDriverWait(driver, 20)

        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'image-slider__main-image')))        
        hotel1_attributes = driver.find_element(By.XPATH, '/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[1]/div[2]/div[3]/div[4]/theme-button/button').get_attribute('outerHTML')
        attributes_filter1 = ''.join(hotel1_attributes.partition('id')[1:])
        attributes_filter2 = attributes_filter1.find('type')
        attributes_filter3 = attributes_filter1[:attributes_filter2]
        attributes_replacers = attributes_filter3.replace('"','').replace('=','-').replace(' ','').replace('id-','')
        query = "'#" + attributes_replacers + "'"
        
        driver.execute_script(f"document.querySelector({query}).click();")
        new_window = driver.window_handles[1]
        driver.switch_to.window(new_window)
        
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'rooms-details-and-price__button')))
        driver.execute_script("document.querySelector('#hotel-details-and-info > div > rooms-details-and-price > div > div.rooms-details-and-price__prices-select-holder > theme-button > button').click();")
        
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-checkout-prices-and-tax')))        
        price_net_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[2]/span[2]')
        tax_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[3]/span[2]')

        price_net_temp.append(price_net_elem.text)
        tax_temp.append(tax_elem.text)
        price_net = rate_fmt(price_net_temp)
        tax = rate_fmt(tax_temp)

        calc_fee = tax / price_net
        fee = float("%.4f" % calc_fee)        

      except:
        c_dest += 1

    return fee


get_fee()


0.1446

In [ ]:
# Scraper

class Scraper_123milhas_Decolar:

    def scraper(self):
        self.data_collect()

    def data_collect(self):

        c_date = 0
        while True:
          try:
            checkin = dates["checkin"][c_date]
            checkout = dates["checkout"][c_date]
          
            c_dest = 0
            while True:
              try:
                name_destination = destinations["name_destination"][c_dest]
                id_destination_123milhas = destinations["id_destination_123milhas"][c_dest]
                id_destination_decolar = destinations["id_destination_decolar"][c_dest]

                list_name_hotels_123milhas = []
                list_price_hotels_123milhas = []
                list_channel_hotels_123milhas = []

                list_name_hotels_decolar = []
                list_price_hotels_decolar = []

                # Scraper 123 Milhas
                self.list_names = []
                self.list_prices = []
                self.list_channels = []
                self.driver = webdriver.Chrome(options=chrome_options)
                self.link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
                self.driver.get(self.link)
                sleep(3)
                wait = WebDriverWait(self.driver, 20)

                for p in range(20): # Number of pages to browse (nº > total, run through all)
                    item = 1
                    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-list-card__total-price')))
                    list_length = self.driver.find_elements(By.CLASS_NAME, 'hotel-list-card__total-price')
                    list_max = len(list_length)
                    for i in range(list_max):
                        c = 1
                        while c < list_max:
                            try:
                                names_temp = self.driver.find_elements(By.XPATH,
                                    f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/div/h4')
                                self.list_names.append(names_temp[0].text)                       
                                sleep(1)
                                prices_temp = self.driver.find_elements(By.XPATH,
                                    f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/div[1]/div/div')
                                self.list_prices.append(prices_temp[0].text)          
                                sleep(1)
                                channels_temp = self.driver.find_element(By.XPATH,
                                    f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/image-slider/div/div[1]/img').get_attribute('outerHTML')
                                if len(channels_temp) == 0:
                                    channels_temp = self.driver.find_element(By.XPATH, f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/image-slider/div/div[1]/img').get_attribute('outerHTML')                        
                                if 'omnibees' in channels_temp:
                                    self.list_channels.append(1)
                                else:
                                    self.list_channels.append(0)
                                sleep(1)
                                item += 1
                            except:
                                item += 1
                                c += 1                        
                    try:
                      wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'pagination-next')))
                      self.driver.execute_script('document.querySelectorAll("#hotelListHolder > div.hotel-card-price-list__cards-holder.hotel-card-price-list__obj-align > div.hotel-card-price-list__pagination-container > ul > li.pagination-next > a")[0].click();')
                      print(f'\u001b[32m{"Browsing..."}\u001b[0m')
                      sleep(3)
                    except:
                      self.driver.quit()

                print('Raw Data:')
                print(self.list_names)
                print(self.list_prices)
                print(self.list_channels)

                for name in self.list_names:
                    list_name_hotels_123milhas.append(name)
                for price in self.list_prices:
                    list_price_hotels_123milhas.append(price)  
                for channel in self.list_channels:
                    list_channel_hotels_123milhas.append(channel)

                # Data processing
                name_hotels_123m_temp = list(pd.Series(list_name_hotels_123milhas).str.upper())
                name_hotels_123m_fmtg = []

                for name in name_hotels_123m_temp:
                    name_hotels_123m_fmtg.append(unidecode(name))

                price_hotels_123m_fmtg = []

                for price in list_price_hotels_123milhas:
                    for dot in ["."]:
                        price = price.replace(dot, "")
                    price_hotels_123m_fmtg.append(price)

                price_hotels_123m_fmtg2 = []

                for price in price_hotels_123m_fmtg:
                    for sign in ["R$ "]:
                        price = price.replace(sign, "")
                    price_hotels_123m_fmtg2.append(price)        

                price_hotels_123m_fmtg2 = pd.to_numeric(price_hotels_123m_fmtg2, errors='coerce', downcast='unsigned')

                data_hotels_123m = []

                for elems in zip(name_hotels_123m_fmtg, price_hotels_123m_fmtg2):
                    data_hotels_123m.append(elems)
                data_hotels_123m_dict = dict(data_hotels_123m)

                df_data_hotels_123m = pd.DataFrame(list(data_hotels_123m_dict.items()),
                                  columns=['Hotel Name', 'Net Price 123 Milhas'])

                length_df1 = len(df_data_hotels_123m)
                length_df2 = len(list_channel_hotels_123milhas)
                diff_length_dfs = length_df2 - length_df1
                list_channel_hotels_123milhas = list_channel_hotels_123milhas[:length_df2 - diff_length_dfs]

                df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'] = (df_data_hotels_123m['Net Price 123 Milhas'] * fee) + df_data_hotels_123m['Net Price 123 Milhas'].apply(pd.to_numeric) 
                df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'] = df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'].astype(int)
                df_data_hotels_123m.insert(loc=1, column='Omnibees Provider 123 Milhas', value=list_channel_hotels_123milhas)

                # Scraper Decolar
                checkin_decolar = datetime.strptime(checkin, "%d-%m-%Y").strftime('%Y-%m-%d')
                checkout_decolar = datetime.strptime(checkout, "%d-%m-%Y").strftime('%Y-%m-%d')

                page = 1      
                self.driver = webdriver.Chrome(options=chrome_options)
                self.link = f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}'
                self.list_names = []
                self.list_prices = []
                self.driver.get(self.link)
                self.driver.find_element(By.XPATH, '//*[@id="lgpd-banner"]/div/a[2]').click()
                sleep(3)
                
                for p in range(20): # Number of pages to browse (nº > total, run through all)
                    item = 1
                    list_length = self.driver.find_elements(By.CLASS_NAME, 'accommodation-name')
                    list_max = len(list_length)
                    for i in range(list_max):
                        c = 1
                        while c < list_max:
                            try:
                                names_temp = self.driver.find_elements(By.XPATH,
                                    f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[1]/div/div[2]/div/aloha-cluster-accommodation-info-container/div[1]/span')
                                self.list_names.append(names_temp[0].text)
                                sleep(1)
                                prices_temp = self.driver.find_elements(By.XPATH,
                                    f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[2]/aloha-cluster-pricebox-container/div/div[2]/div[1]/aloha-price-container/aloha-summary-price-test/div/span[2]')
                                self.list_prices.append(prices_temp[0].text)          
                                sleep(1)
                                item += 1
                            except:
                                item += 1
                                c += 1                        
                    try:
                        page += 1
                        self.driver.get(f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}')
                        print(f'\u001b[32m{"Browsing..."}\u001b[0m')
                        sleep(5)
                    except:
                        self.driver.quit()
                
                print("Raw data:")
                print(self.list_names)
                print(self.list_prices)

                for name in self.list_names:
                    list_name_hotels_decolar.append(name)
                for price in self.list_prices:
                    list_price_hotels_decolar.append(price)
                
                # Data processing
                name_hotels_desp_temp = list(pd.Series(list_name_hotels_decolar).str.upper())
                name_hotels_desp_fmtg = []

                for name in name_hotels_desp_temp:
                    name_hotels_desp_fmtg.append(unidecode(name))

                price_hotels_desp_fmtg = []

                for price in list_price_hotels_decolar:
                    for dot in ["."]:
                        price = price.replace(dot, "")
                    price_hotels_desp_fmtg.append(price)

                price_hotels_desp_fmtg = pd.to_numeric(price_hotels_desp_fmtg, errors='coerce', downcast="integer")

                data_hotels_desp = []

                for elems in zip(name_hotels_desp_fmtg, price_hotels_desp_fmtg):
                    data_hotels_desp.append(elems)
                data_hotels_desp_dict = dict(data_hotels_desp)
                
                df_data_hotels_desp = pd.DataFrame(list(data_hotels_desp_dict.items()),
                          columns=['Hotel Name', 'Total Price Decolar'])
                
                # Data merge
                data_merge = pd.merge(df_data_hotels_123m, df_data_hotels_desp, on=['Hotel Name'], how='left')
                df_rate_comparison = data_merge.dropna()
                for prices in df_rate_comparison:
                  diff = (df_rate_comparison.loc[:, 'Total Price Decolar'] / df_rate_comparison.loc[:, 'Total Price 123 Milhas (w/ fee)'] -1 ) * 100
                  df_rate_comparison.loc[:, 'Variation (total prices)'] = diff
                
                # Columns additions and transformations 
                # Name
                df_rate_comparison['Hotel Name'] = df_rate_comparison['Hotel Name'].str.title()

                # Price Decolar format
                df_rate_comparison['Total Price Decolar'] = df_rate_comparison['Total Price Decolar'].astype(int)

                # Fee
                fee_round = round(fee, 2)
                fee_fmtg = str(fee_round).replace('.', ',')
                df_rate_comparison['Fee 123 Milhas'] = fee_fmtg
                df_rate_comparison.insert(2, 'Fee 123 Milhas', df_rate_comparison.pop('Fee 123 Milhas'))

                # BML
                df_rate_comparison['BML'] = np.where(df_rate_comparison['Variation (total prices)'] >= 1, 'LOSE', np.where(df_rate_comparison['Variation (total prices)'] <= -1, 'BEAT', 'MEET'))

                # Variation
                df_rate_comparison['Variation (total prices)'] = round(df_rate_comparison['Variation (total prices)'], 2)
                df_rate_comparison['Variation (total prices)'] = df_rate_comparison['Variation (total prices)'].astype(str).str.replace('.',',')

                # Extraction Date
                current_date = date.today().strftime("%d-%m-%Y")
                df_rate_comparison['Extraction Date'] = current_date

                date_format = "%d-%m-%Y"
                cin = datetime.strptime(checkin, date_format)
                cout = datetime.strptime(checkout, date_format)
                
                # ADVP
                today = datetime.strptime(current_date, date_format)
                delta = cin - today
                advp = delta.days
                df_rate_comparison['ADVP'] = advp
                df_rate_comparison.insert(1, 'ADVP', df_rate_comparison.pop('ADVP'))
                
                # LOS
                delta = cout - cin
                los = delta.days
                df_rate_comparison['LOS'] = los
                df_rate_comparison.insert(1, 'LOS', df_rate_comparison.pop('LOS'))    

                # Check-out date
                df_rate_comparison['Check-out date'] = checkout
                df_rate_comparison.insert(1, 'Check-out date', df_rate_comparison.pop('Check-out date'))
                
                # Check-in date
                df_rate_comparison['Check-in date'] = checkin
                df_rate_comparison.insert(1, 'Check-in date', df_rate_comparison.pop('Check-in date'))

                # Destination
                df_rate_comparison['Destination'] = name_destination
                df_rate_comparison.insert(1, 'Destination', df_rate_comparison.pop('Destination'))

                # Upload to Google Sheets
                ws = gc.open("Pricing BR 123 Milhas").worksheet("data")
                ws.add_rows(df_rate_comparison.shape[0])
                gd.set_with_dataframe(worksheet=ws, dataframe=df_rate_comparison, include_index=False, include_column_header=False, row=ws.row_count+1, resize=False)

                c_dest += 1
              except:
                c_dest += 1
                if c_dest > len(destinations["name_destination"]):
                  break
            c_date += 1
          except:
            c_date += 1
            if c_date > len(dates["checkin"]):
              break                


start = Scraper_123milhas_Decolar()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw Data:
['Simas Praia Hotel', 'Celi Hotel Aracaju', 'Arcus Hotel Aracaju (Ant. Comfort Aracaju)', 'Del Mar Hotel - Aracaju', 'Via Mar Praia Hotel', 'Quality Hotel Aracaju', 'Hotel da Costa By Nobile', 'NB Hotéis', 'Aruanã Eco Praia Hotel', 'Radisson Hotel Aracaju', 'Real Classic Hotel', 'Go Inn Aracaju', 'Hotel Pousada do Sol', 'Farol Plaza Hotel', 'Tropical Mar Hotel', 'Jatobá Praia Hotel', 'Pousada Águas Douradas', 'Del Canto Hotel', 'Pousada Encantare', 'Atalaia Apart Hotel', 'Ibis Aracaju Jardins', 'SAN MANUEL PRAIA HOTEL', 'Hotel Terra do Sol', 'Araras Praia Hotel', 'Ocean Hotel', 'Hotel Algas Marinhas', 'Alto da Praia Hotel', 'Nascimento Praia Hotel', 'Hotel Jangadeiro Aracaju', 'Pousada Sao Luiz - Aracaju', 'Hotel Praia e Mar', 'Hotel Aracaj

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:231: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


Browsing...
